# Strategies and Panel Data

When working with panel data, we have two options.

1. We can apply a strategy to all stocks in the panel data.

1. We can subset a particular stock, effectively obtaining time series data for that single asset. From here we can use the same method we did in the previous notebook. This has the advantage of parameter optimisation, which we'll see at the end of this notebook.

Let's start with the *one strategy for all stocks* approach, applying the SMA crossover strategy.

First, imports and preparing the dataframe.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.style.use("ggplot")

In [ ]:
df = pd.read_csv("tech_six.csv")
df["datadate"] = pd.to_datetime(df["datadate"], dayfirst=True)

## Multiple Stocks

Whenever we're going to be repeating an operation, it is usually a good idea (and sometimes necessary) to create our own function. Let's create a function to calculate SMAs and generate trading positions.

In [ ]:

def calculate_sma(df, short_window=50, long_window=200):
    df["fastsma"] = df["prccd"].rolling(window=short_window).mean()
    df["slowsma"] = df["prccd"].rolling(window=long_window).mean()

    df["signal"] = np.where(df["fastsma"] > df["slowsma"], 1, -1)
    df["signal"] = np.where(df["fastsma"].isna(), 0, df["signal"])

    df["position"] = df["signal"].shift()
    return df


With the function done, we can apply it groupwise to the stocks in our panel data.

In [ ]:
# Create a grouping, iterate through and apply our function, then concatenate
grouped = df.groupby("tic")

groups = []
for name, group in grouped:
    groups.append(calculate_sma(group))

df = pd.concat(groups)

Then it's just a matter of calculating the cumulative returns as we've done before

In [ ]:
# Calculate the market return and strategy return
df["mktret"] = df.groupby("tic")["prccd"].pct_change()
df["stratret"] = df["position"] * df["mktret"]

# Calculate the cumulative market and strategy return
df["cmktret"] = (1 + df["mktret"]).groupby(df["tic"]).cumprod()
df["cstratret"] = (1 + df["stratret"]).groupby(df["tic"]).cumprod()

df.groupby("tic")[["cmktret", "cstratret"]].last()
